In [1]:
import pandas as pd
from pymongo import MongoClient
import json
import numpy as np
import geopandas
import sys
import matplotlib.pyplot as plt
import os
import requests
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap

In [2]:
def connectMongoDb(): 
    dbName = "datamad0620"
    mongodbURL = f"mongodb://localhost/{dbName}"

    client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
    db = client.get_database()
connectMongoDb()

In [3]:
dbName = "datamad0620"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()
    
    
def companiesMongo_count(country_code):
    query = {"offices.country_code":f"{country_code}"}
    
    res = list(db.companies.find(query))
    return len(res) 

companiesMongo_count("SGP")

84

In [4]:
def companiesMongo(country_code):
    query = {"offices.country_code":f"{country_code}"}
    
    projection ={"offices":1, "name":1, "founded_year":1,"total_money_raised":1, "category_code":1}
    res = list(db.companies.find(query,projection))
    return res

result= companiesMongo("SGP")
r=pd.DataFrame(result).sort_values(by="category_code", ascending=False).head(100)
r.groupby('category_code').agg({'category_code':"count"})


,category_code
category_code,
advertising,5
analytics,1
biotech,1
cleantech,1
consulting,3
ecommerce,2
enterprise,7
finance,1
games_video,2


In [5]:
def limipiezaData(companies,code):
    df = pd.DataFrame(list(companies))
    offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
    clean_offices = pd.concat([df[["name","_id"]], offices], axis=1)
    clean_offices = clean_offices.rename(columns={"_id":"company_id"})
    filtered = clean_offices[(clean_offices['country_code']== f"{code}")]
    return filtered   

data = limipiezaData(result, "SGP")

In [6]:
import numpy as np

def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }
    

data["geopoint"] = data.apply(transformToGeoPoint, axis=1)

In [7]:
data=data.dropna(subset=['longitude'])
data=data.dropna(subset=['latitude'])


In [8]:
m = Map(location=[1.290270,103.851959],zoom_start=14)
for i, row in data.iterrows():
    Marker(
        location=row[["latitude","longitude"]],
        tooltip=row["name"],
        popup=row["address1"]
    ).add_to(m)
m

In [9]:
def requestFoursquare(query,radio):
    
    client_s = os.getenv("CLIENT_SECRET")
    client_idd = os.getenv("4S_CLIENT_ID")
    lat = 1.298370
    long = 103.891290

    params = {"client_id": client_idd,
              "client_secret":"LNVYSPVLUTJITTBVZ1DHWMWISC43K43N1DTACC5CYWFYRRCZ",
              "v": "20180323",
              "ll":f'{lat},{long}',
              "radius":f'{radio}',
              "query":f'{query}', 
              "limit":100    }

    url = f"https://api.foursquare.com/v2/venues/explore"
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    print(url)
    return data["response"]

requestFoursquare("restaurant",3000)

https://api.foursquare.com/v2/venues/explore


{}

### ponderacion

In [10]:
total = 87
training_w=87/total
school_w=26/total
party_w=50/total
starbucks_w=9/total
vegan=1/total
basketbal=1/total
dog=1/total

- Traning: Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- Work-life balance: 30% of the company have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
- Account managers need to travel a lot
- All people in the company have between 25 and 40 years, give them some place to go to party.
- The CEO is Vegan
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog "Pepe" needs a hairdresser every month. Ensure there's one not too far away


In [11]:
def requestFoursquare(query,radio):
    client_idd = os.getenv("4S_CLIENT_ID")
    client_secret=os.getenv("CLIENT_SECRET")
    lat = 1.298370
    long = 103.891290

    params = {"client_id": client_idd,
              "client_secret":client_secret,
              "v": "20180323",
              "ll":f'{lat},{long}',
              "radius":f'{radio}',
              "query":f'{query}', 
              "limit":20 }

    url = f"https://api.foursquare.com/v2/venues/explore"
    
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    
    return data
requestFoursquare("coffe",2000)

{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5efa740e4348d913c6eb2e76'},
 'response': {}}

In [19]:
import googlemaps
from dotenv import load_dotenv
load_dotenv()

def gPlaces (lat,long,query,rad=1000):
    apiKey = os.getenv('GPLACES_APIKEY')

    gmaps =googlemaps.Client(key=apiKey)
    
    places = gmaps.places_nearby(location=[lat,long], radius=rad,open_now=False, type=query)
    return places["results"]

    #places = gmaps.places_nearby(location=[1.298370,103.891290], radius=rad,open_now=False, type=query)


In [21]:
gPlaces(1.298370,103.891290,"coffe",1000)


[{'geometry': {'location': {'lat': 1.3553794, 'lng': 103.8677444},
   'viewport': {'northeast': {'lat': 1.4708809, 'lng': 104.0415799},
    'southwest': {'lat': 1.216611, 'lng': 103.6065099}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
  'id': 'de9886df6eae88551e41600d0927dc8eb9f2c6ca',
  'name': 'Singapore',
  'photos': [{'height': 853,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108093408089432966223">Adam Bolt</a>'],
    'photo_reference': 'CmRaAAAAxR6dzdPhMzIfsLlj2zyOee4IVO-aQJsXpE6uzunR3YLF0IWripm8yiXP3hzK7qSGp1RPrIGcS0fMUTpxpQ9FTU6YzKiUv4BLNBIJmWhk20I8fRCC3uONmVMaJ-XG3A-8EhAABebousC7mCypwIEBxgVTGhSJaqJp0QFkJQvAdeNLM_vsU5hVaw',
    'width': 1280}],
  'place_id': 'ChIJyY4rtGcX2jERIKTarqz3AAQ',
  'reference': 'ChIJyY4rtGcX2jERIKTarqz3AAQ',
  'scope': 'GOOGLE',
  'types': ['locality', 'political'],
  'vicinity': 'Singapore'},
 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 1.298376, 'lng': 103.886

In [ ]:
results_map=[(places[i]["name"], places[i]["geometry"]["location"]["lat"], places[i]["geometry"]["location"]["lng"]) for i in range(len(places))]
results_map=pd.DataFrame(results_map, columns=['name', 'latitude', 'longitude'])
results_map

In [ ]:
from shapely.geometry import Point, Polygon
import geopandas as gpd
import descartes
%matplotlib inline

geometry=[Point(xy) for xy in zip(results_map["latitude"], results_map["longitude"])]
geo_df=gpd.GeoDataFrame(results_map,geometry=geometry)
geo_df

In [ ]:
geo_df=geo_df.dropna(subset=['longitude'])
geo_df=geo_df.dropna(subset=['latitude'])

In [ ]:
m = Map(location=[1.290270,103.851959],zoom_start=14)
for i, row in geo_df.iterrows():
    icon = Icon(
                color="red",
                prefix="fa",
                icon="trophy"
            )
    Marker(
        location=row[["latitude","longitude"]],
        tooltip=row["name"],
    ).add_to(m)
m